## basics to get everything to work
[Learn how to find your Qualtrics IDs](https://www.qualtrics.com/support/integrations/api-integration/finding-qualtrics-ids/)

In [16]:
# ---- import packages
import pandas as pd
import numpy as np
import json
import os
import requests
import base64
from io import StringIO
import time

## Let's download data from the Qualtrics cloud (aka their API); this has a number of benefits:
#### 1. My data aren't stuck in my Qualtrics account; I can provide my Qualtrics IDs and anyone can access it.

In [17]:
qualtrics_data_center = 'YOUR QUALTRICS DATA CENTER'
api_token = 'YOUR QUALTRICS API TOKEN'
survey_id = 'YOUR QUALTRICS SURVEY ID'

#### 2. I can import the data right into the my notebook
#### 3. I define export settings that make the raw data most interpretable
#### 4. The Qualtrics website is sloowww and using it for anything other than building surveys can be frustrating
---

In [18]:
# ---- settings for the survey export
export_settings = {'format':'csv','useLabels':True,'breakoutSets':False,'compress':False}

# ---- export the survey and provide the progress ID
url = 'https://{}/API/v3/surveys/{}/export-responses'.format(qualtrics_data_center, survey_id)
export_survey = requests.post(url, json = export_settings, headers={'x-api-token': api_token, 'content-type': 'application/json'}).text
progressId = json.loads(export_survey)['result']['progressId']

# ---- using the progress ID, get the file ID for the exported survey data
url = 'https://{}/API/v3/surveys/{}/export-responses/{}'.format(qualtrics_data_center, survey_id, progressId)
get_fileId = requests.get(url, headers={'x-api-token': api_token, 'content-type': 'application/json'}).text

# ---- if the export isn't done yet, check back in 5 seconds
while json.loads(get_fileId)['result']['status'] == 'inProgress':
    print('data downloading from Qualtrics API...')
    time.sleep(5)
    get_fileId = requests.get(url, headers={'x-api-token': api_token, 'content-type': 'application/json'}).text
fileId = json.loads(get_fileId)['result']['fileId']

# ---- once I have the file ID, get the exported data
url = 'https://{}/API/v3/surveys/{}/export-responses/{}/file'.format(qualtrics_data_center, survey_id, fileId)
get_export = requests.get(url, headers={'x-api-token': api_token, 'content-type': 'application/json'})
raw_data = pd.read_csv(StringIO(get_export.text), header=1)
raw_data = raw_data.drop(0)

# ---- show the top five lines of the raw data
raw_data.head()

,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,employment_status,age,gender_identity,market,email,searches_in_app_30_days,monthly_active_user,device_type,sessions_in_app_30_days,account_type
1,2020-02-24 02:11:00,2020-02-24 02:11:25,IP Address,98.26.39.12,100,25,True,2020-02-24 02:11:26,R_BAqBRhrMpLXM0Rb,NaN,...,stay_at_home_parent,35_39,female,Cananda,example_email0@outlook.com,1,1,android,1,paid
2,2020-02-24 02:11:39,2020-02-24 02:11:49,IP Address,98.26.39.12,100,10,True,2020-02-24 02:11:49,R_1dalvbvejfbrRAg,NaN,...,student,18_24,non_binary,US,example_email1@icloud.com,0,1,android,17,free
3,2020-02-24 02:11:57,2020-02-24 02:12:08,IP Address,98.26.39.12,100,11,True,2020-02-24 02:12:09,R_3a8UdJqI6iF1bz3,NaN,...,employed_full_time,30_34,female,US,example_email2@outlook.com,4,1,android,4,paid
4,2020-02-24 02:12:15,2020-02-24 02:12:29,IP Address,98.26.39.12,100,14,True,2020-02-24 02:12:30,R_1eyiw27iv2tBynH,NaN,...,unemployed_retired,50_55,male,Cananda,example_email3@icloud.com,0,1,apple,2,paid
5,2020-02-24 02:12:37,2020-02-24 02:12:58,IP Address,98.26.39.12,100,20,True,2020-02-24 02:12:58,R_R4B5BljazoqtFKN,NaN,...,student,25_29,male,US,example_email4@gmail.com,0,0,android,0,paid


---
## Qualtrics data exports are not pretty; let's get rid of extra stuff that Qualtrics adds on

In [19]:
# ---- export the raw data and created a copy to edit
raw_data.to_excel(os.path.abspath('data/raw_data.xlsx'))
cleaned_data = raw_data.copy()

# ---- get rid of all of the Qualtrics output extras
cleaned_data = cleaned_data.rename(columns={'Response ID': 'response_id'}).set_index('response_id').iloc[:, 16:]

# ---- remove text that Qualtrics appends to headers
cleaned_data.columns = [header.replace(' - Selected Choice', '').replace(' - Other (please specify) - Text', '_other') for header in cleaned_data.columns]

# ---- convert strings to integers
for column in ['searches_in_app_30_days', 'monthly_active_user', 'sessions_in_app_30_days']:
    cleaned_data[column] = cleaned_data[column].astype(int)
    
# ---- fix a typo!
cleaned_data.market = np.where(cleaned_data.market == 'Cananda', 'Canada', cleaned_data.market)
    
# ---- show the top five lines of the cleaned data
cleaned_data.head()

,satisfaction_spotify,devices_past_week,devices_past_week_other,employment_status,age,gender_identity,market,email,searches_in_app_30_days,monthly_active_user,device_type,sessions_in_app_30_days,account_type
response_id,,,,,,,,,,,,,
R_BAqBRhrMpLXM0Rb,somewhat_satisfied,"game_console,car,other",other example,stay_at_home_parent,35_39,female,Canada,example_email0@outlook.com,1,1,android,1,paid
R_1dalvbvejfbrRAg,neither,mobile_phone,NaN,student,18_24,non_binary,US,example_email1@icloud.com,0,1,android,17,free
R_3a8UdJqI6iF1bz3,very_dissatisfied,tablet,NaN,employed_full_time,30_34,female,US,example_email2@outlook.com,4,1,android,4,paid
R_1eyiw27iv2tBynH,very_satisfied,"mobile_phone,game_console,tv,car,smart_speaker",NaN,unemployed_retired,50_55,male,Canada,example_email3@icloud.com,0,1,apple,2,paid
R_R4B5BljazoqtFKN,very_satisfied,"mobile_phone,computer,tv,car,smart_speaker",NaN,student,25_29,male,US,example_email4@gmail.com,0,0,android,0,paid


---
## Now that I have clean data, I want to flag users that are not eligible to participate in my study

In [20]:
# ---- create flags for eligibility and sort data
cleaned_data['exclude'] = np.nan
cleaned_data.loc[cleaned_data[cleaned_data.age == 'under_18'].index, 'exclude'] = 'under_18'
cleaned_data.loc[cleaned_data[cleaned_data.market == 'Canada'].index, 'exclude'] = 'wrong_market'
cleaned_data.loc[cleaned_data[cleaned_data.device_type == 'apple'].index, 'exclude'] = 'not_android_user'

cleaned_data = cleaned_data.sort_values('exclude')

# ---- export cleaned data
cleaned_data.to_excel(os.path.abspath('data/cleaned_data.xlsx'))

# ---- show the top five lines of the cleaned data
cleaned_data

,satisfaction_spotify,devices_past_week,devices_past_week_other,employment_status,age,gender_identity,market,email,searches_in_app_30_days,monthly_active_user,device_type,sessions_in_app_30_days,account_type,exclude
response_id,,,,,,,,,,,,,,
R_1eyiw27iv2tBynH,very_satisfied,"mobile_phone,game_console,tv,car,smart_speaker",NaN,unemployed_retired,50_55,male,Canada,example_email3@icloud.com,0,1,apple,2,paid,not_android_user
R_2qdEMaZwuBgFMUy,somewhat_satisfied,computer,NaN,self_employed_contractor,55_plus,prefer_not_to_say,US,example_email8@icloud.com,24,1,apple,12,free,not_android_user
R_2qvHuQcF7xiB09e,somewhat_satisfied,"tv,car",NaN,student,under_18,other,US,example_email5@gmail.com,0,1,android,1,free,under_18
R_2tiqcW9LkkcIhHt,very_satisfied,"mobile_phone,computer,game_console,tv",NaN,student,under_18,male,US,example_email6@gmail.com,6,1,android,6,free,under_18
R_BAqBRhrMpLXM0Rb,somewhat_satisfied,"game_console,car,other",other example,stay_at_home_parent,35_39,female,Canada,example_email0@outlook.com,1,1,android,1,paid,wrong_market
R_1QMhLx313cOXg4Z,neither,tv,NaN,self_employed_business_owner,45_49,female,Canada,example_email7@yahoo.com,6,1,android,3,paid,wrong_market
R_1dalvbvejfbrRAg,neither,mobile_phone,NaN,student,18_24,non_binary,US,example_email1@icloud.com,0,1,android,17,free,NaN
R_3a8UdJqI6iF1bz3,very_dissatisfied,tablet,NaN,employed_full_time,30_34,female,US,example_email2@outlook.com,4,1,android,4,paid,NaN
R_R4B5BljazoqtFKN,very_satisfied,"mobile_phone,computer,tv,car,smart_speaker",NaN,student,25_29,male,US,example_email4@gmail.com,0,0,android,0,paid,NaN
